In [1]:
from random import random, randint
from sklearn.ensemble import RandomForestRegressor

import mlflow
import mlflow.sklearn

mlflow.set_experiment("Model Registry")

with mlflow.start_run(run_name="first run") as run:
    params = {"n_estimators": 5, "random_state": 42}
    sk_learn_rfr = RandomForestRegressor(**params)

    # Log parameters and metrics using the MLflow APIs
    mlflow.log_params(params)
    mlflow.log_param("param_1", randint(0, 100))
    mlflow.log_metrics({"metric_1": random(), "metric_2": random() + 1})

    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=sk_learn_rfr,
        artifact_path="sklearn-model",
        registered_model_name="random-forest-model"
    )

Registered model 'random-forest-model' already exists. Creating a new version of this model...
2022/04/28 12:32:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-model, version 3
Created version '3' of model 'random-forest-model'.


In [2]:
run_names = ["second run", "third run", "fourth run"]

for rn in run_names:
    with mlflow.start_run(run_name=rn) as run:
        params = {"n_estimators": 5, "random_state": 42}
        sk_learn_rfr = RandomForestRegressor(**params)

        # Log parameters and metrics using the MLflow APIs
        mlflow.log_params(params)
        mlflow.log_param("param_1", randint(0, 100))
        mlflow.log_metrics({"metric_1": random(), "metric_2": random() + 1})

        # Log the sklearn model and register as version 1
        mlflow.sklearn.log_model(
            sk_model=sk_learn_rfr,
            artifact_path="sklearn-model"
        )

In [3]:
mlflow.register_model(
    "runs:/d06a7fb203024b88bdb57226bd82fc81/sklearn-model",
    "random-forest-model"
)

Registered model 'random-forest-model' already exists. Creating a new version of this model...
2022/04/28 12:33:22 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-model, version 4
Created version '4' of model 'random-forest-model'.


<ModelVersion: creation_timestamp=1651149202275, current_stage='None', description='', last_updated_timestamp=1651149202275, name='random-forest-model', run_id='d06a7fb203024b88bdb57226bd82fc81', run_link='', source='gs://prod-mlflow-7948/92/d06a7fb203024b88bdb57226bd82fc81/artifacts/sklearn-model', status='READY', status_message='', tags={}, user_id='', version='4'>

In [4]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
client.create_registered_model("random-forest-model-new")

<RegisteredModel: creation_timestamp=1651149202675, description='', last_updated_timestamp=1651149202675, latest_versions=[], name='random-forest-model-new', tags={}>

In [5]:
client.create_model_version(
    name="random-forest-model-new",
    source="gs://prod-mlflow-7948/92/69a0e5fc3cc44ce19e18d08761399623/artifacts/sklearn-model",
    run_id="69a0e5fc3cc44ce19e18d08761399623"
)

2022/04/28 12:33:23 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-model-new, version 1


<ModelVersion: creation_timestamp=1651149203083, current_stage='None', description='', last_updated_timestamp=1651149203083, name='random-forest-model-new', run_id='69a0e5fc3cc44ce19e18d08761399623', run_link='', source='gs://prod-mlflow-7948/92/69a0e5fc3cc44ce19e18d08761399623/artifacts/sklearn-model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [6]:
client.update_model_version(
    name="random-forest-model-new",
    version=1,
    description="This model version is a scikit-learn random forest containing 100 decision trees"
)

<ModelVersion: creation_timestamp=1651149203083, current_stage='None', description=('This model version is a scikit-learn random forest containing 100 decision '
 'trees'), last_updated_timestamp=1651149203479, name='random-forest-model-new', run_id='69a0e5fc3cc44ce19e18d08761399623', run_link='', source='gs://prod-mlflow-7948/92/69a0e5fc3cc44ce19e18d08761399623/artifacts/sklearn-model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [7]:
client.rename_registered_model(
    name="random-forest-model-new",
    new_name="random-forest-model-updated"
)

In [8]:
client.transition_model_version_stage(
    name="random-forest-model",
    version=2,
    stage="Production"
)

<ModelVersion: creation_timestamp=1651148979327, current_stage='Production', description='', last_updated_timestamp=1651149204311, name='random-forest-model', run_id='d06a7fb203024b88bdb57226bd82fc81', run_link='', source='gs://prod-mlflow-7948/92/d06a7fb203024b88bdb57226bd82fc81/artifacts/sklearn-model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [9]:
client.delete_model_version(name="random-forest-model", version=3)

In [10]:
client.delete_registered_model(name="random-forest-model-updated")